<a href="https://colab.research.google.com/github/PacktPublishing/Machine-Learning-for-Time-Series-with-Python/blob/master/chapter6/Change-Points_Anomalies.ipynb" target="_parent\"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anomaly detection

In [1]:
#!pip install alibi_detect

In [2]:
from alibi_detect.datasets import fetch_kdd

intrusions = fetch_kdd()

2025-05-30 13:16:26.599612: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-30 13:16:26.606885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748603786.615008  206044 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748603786.617330  206044 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748603786.623629  206044 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
intrusions["target"].sum() / len(intrusions["target"])

0.8030893423558918

In [4]:
intrusions["feature_names"]

['srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate']

In [6]:
from pyod.models.knn import KNN

X_col = intrusions["data"][:, 0]

od = KNN()
od.fit(X_col.reshape(-1, 1))

scores = od.decision_function(X_col.reshape(-1, 1))  # score d'anomalie
labels = od.predict(X_col.reshape(-1, 1))  # 0 = normal, 1 = anomalie


In [ ]:
import pandas as pd

pd.Series(intrusions["data"][:, 0]).plot();

In [ ]:
from alibi_detect.od import SpectralResidual

od = SpectralResidual(
    threshold=1.0, window_amp=20, window_local=20, n_est_points=10, n_grad_points=5
)
intrusion_outliers = od.predict(intrusions["data"][:,0])

In [ ]:
import matplotlib

ax = pd.Series(intrusions["data"][:, 0], name="data").plot(
    legend=False, figsize=(12, 6)
)
ax2 = ax.twinx()
ax = pd.Series(scores, name="scores").plot(
    ax=ax2, legend=False, color="r", marker=matplotlib.markers.CARETDOWNBASE
)
ax.figure.legend(bbox_to_anchor=(1, 1), loc="upper left");

# Change point detection

In [ ]:
!pip install ruptures

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ruptures as rpt

plt.style.use("seaborn-whitegrid")
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "17"

In [ ]:
signal, bkps = rpt.pw_constant(
    n_samples=500, n_features=3, n_bkps=2, noise_std=5.0, delta=(1, 20)
)

In [ ]:
rpt.display(signal, bkps)

In [ ]:
signal.shape

In [ ]:
bkps

In [ ]:
# "l1", "rbf", "linear", "normal", "ar"
algo = rpt.Binseg(model="l1").fit(signal)
my_bkps = algo.predict(n_bkps=3)

# show results
rpt.show.display(signal, bkps, my_bkps, figsize=(10, 6))